In [1]:
import networks
import pretrainedmodels
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [2]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

#MNIST用のtransform
#transform = transforms.Compose([transforms.ToTensor()])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=500, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=500,shuffle=False, num_workers=2)

#classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [3]:
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

#net = networks.xception(num_classes=10)

net = pretrainedmodels.__dict__["xception"](num_classes=1000, pretrained='imagenet')
#グレースケールなので，第１層だけ1チャンネルのものに書き換える
#net.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=2, bias=False)

In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
net.to(device)

import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001, weight_decay=5e-4)

In [5]:
train_loss_list = []
train_acc_list = []
val_loss_list = []
val_acc_list = []

for epoch in range(30):  # loop over the dataset multiple times

    train_loss_per_epoch = 0.0
    train_loss_per_iter = 0.0
    val_loss_per_epoch = 0.0
    val_loss_per_iter = 0.0
    
    train_acc_per_epoch = 0.0
    train_acc_per_iter = 0.0
    val_acc_per_epoch = 0.0
    val_acc_per_iter = 0.0
    
    net.train()
    
    for i, data in enumerate(trainloader, 0):
        # 第二引数は，スタート位置で，0なのでenumerate(trainloader)と同じ
        # https://docs.python.org/3/library/functions.html#enumerate

        # get the inputs
        inputs, labels = data
        
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        train_loss_per_epoch += loss.data
        train_loss_list.append(loss.data/500)
    #print('[%d] loss: %.3f' % (epoch + 1, train_loss_per_epoch / 60000))
    
    net.eval()
    
    for i, data in enumerate(testloader, 0):
        # get the inputs
        inputs, labels = data
        
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        outputs = net(inputs)
        pred = outputs.argmax(1).to("cpu")
        
        
        
        loss = criterion(outputs, labels)
        
        # print statistics
        val_loss_per_epoch += loss.data
        val_loss_list.append(loss.data)
        val_acc_per_epoch += (pred==labels.to("cpu")).sum().item()
        val_acc_list.append((pred==labels.to("cpu")).sum().item())
        
    print('[%d] test_acc: %.3f' % (epoch + 1, val_acc_per_epoch / 10000))
    
print('Finished Training')

[1] test_acc: 0.727
[2] test_acc: 0.770
[3] test_acc: 0.803
[4] test_acc: 0.798
[5] test_acc: 0.798
[6] test_acc: 0.812
[7] test_acc: 0.816
[8] test_acc: 0.823
[9] test_acc: 0.813
[10] test_acc: 0.815
[11] test_acc: 0.826
[12] test_acc: 0.818
[13] test_acc: 0.812
[14] test_acc: 0.823
[15] test_acc: 0.822
[16] test_acc: 0.823
[17] test_acc: 0.824
[18] test_acc: 0.818
[19] test_acc: 0.811
[20] test_acc: 0.834
[21] test_acc: 0.827
[22] test_acc: 0.822
[23] test_acc: 0.820
[24] test_acc: 0.824
[25] test_acc: 0.818
[26] test_acc: 0.818
[27] test_acc: 0.819
[28] test_acc: 0.830
[29] test_acc: 0.804
[30] test_acc: 0.820
Finished Training


In [11]:
net

Xception(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (block1): Block(
    (skip): Conv2d(64, 128, kernel_size=(1, 1), stride=(2, 2), bias=False)
    (skipbn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (rep): Sequential(
      (0): SeparableConv2d(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
        (pointwise): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      )
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): SeparableConv2d(
        (conv1): Co